## **Configuring the account**

In [ ]:
import subprocess, gc, os
# identifying into github
!git config --global user.name "Nuzz23"
!git config --global user.email "nunzio.licalzi9@gmail.com"
output = subprocess.check_output("git config --global --list", shell=True).decode('utf-8').split()

# check if correctly identified
assert len(output) >= 2, "Wrong lenght"
assert output[0].split('=')[-1] == 'Nuzz23', 'wrong user name'
assert output[1].split('=')[-1] == 'nunzio.licalzi9@gmail.com', 'wrong email'
del output

In [ ]:
with open("TOKENS.txt", 'r', encoding='utf-8') as f:
  for line in f:
    os.environ[line.split('=')[0].strip()] = line.strip().split('=')[1].strip()

## **Cloning the repository**

In [ ]:
# if the cloned repository already exists delete it
%cd /content
%rm -rf sample_data
if "MLDL_SemanticSegmentation" in subprocess.check_output("ls", shell=True).decode("utf-8").strip():
  !rm -rf MLDL_SemanticSegmentation

/content


In [ ]:
# clone the repository via token
!git clone --quiet "{os.getenv("GITHUB_TOKEN")}"

# check if cloned correctly
assert "MLDL_SemanticSegmentation" in subprocess.check_output("ls", shell=True).decode("utf-8").strip(), "Not cloned correctly"

# **LIBRARIES**

In [ ]:
# installing the required libraries
%cd /content/MLDL_SemanticSegmentation
!pip install -q -r requirements.txt

/content/MLDL_SemanticSegmentation
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━

## **CUSTOM IMPORTS**

In [ ]:
from stats import countFLOPS, latency, evaluateLastEpoch
from train.trainDACS import init_model
from datasets.downloader import Downloader

# **DATASET**

## **Downloading the dataset**

In [ ]:
if not Downloader().downloadCityScapes():
  raise FileNotFoundError("CityScapes dataset not found")

In [ ]:
if not Downloader().downloadGTA5():
  raise FileNotFoundError("GTAV dataset not found")

# **MODEL**

In [ ]:
from datasets.dataAugmentation.horizontalFlip import HorizontalFlip
from datasets.dataAugmentation.saltAndPepper import SaltAndPepper
from datasets.dataAugmentation.gaussianBlur import GaussianBlur
from datasets.dataAugmentation.colorJitter import ColorJitter

In [ ]:
TRAIN_SIZE, VAL_SIZE = (1280, 720), (1024, 512)

model = init_model(model_str='bisenet', batchSize=4, learning_rate=5e-4, trainSize=TRAIN_SIZE, valSize=VAL_SIZE, momentum=0.9,
                   augmentation=[ColorJitter(p=0.50, brightness=0, contrast=0, saturation=0, hue=0.1)],
                   enablePrint=True, pushWeights=True, enablePrintVal=True, restartTraining=True)

flops = countFLOPS(model, width=VAL_SIZE[0], height=VAL_SIZE[1])
latencyMean, latencyStd, fpsMean, fpsStd = latency(model, width=VAL_SIZE[0], height=VAL_SIZE[1])
evaluateLastEpoch(model)